<a href="https://colab.research.google.com/github/macintoxic/BootCampRendaVariavel/blob/main/trabalho_pratico_2/trabalho_pratico_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instala as dependencias

In [ ]:
!pip install pandas_datareader --upgrade
!pip install sidrapy

## Imports

In [51]:

import yfinance as yf
import requests
import pandas as pd
import sidrapy as sidra
import sqlite3
from pandas_datareader import data
from sqlalchemy import create_engine 



## Colete a série histórica dos dados de inflação de bebidas do IBGE usando a API sidra

In [15]:
url_ipca = 'https://apisidra.ibge.gov.br/values/t/1705/n1/all/v/355/p/all/c315/7169,7170,7389,7396,7397,7440,7443,12394/d/v355%202'

ipca_raw = sidra.get_table(table_code=1705,
                           territorial_level = "1",
                           ibge_territorial_code = "all",
                           period = "all",
                           categories=[7169,7170,7389,7396,7397,7440,7443,12394]                           
                           )


ipca_raw.head()



## Colete a série histórica dos dados diários de preços e volume
(OHLCV) para a ação ABEV3 do provedor de sua preferência (yahoo
finance, marketstack, eodhistoricaldata, finnhub, alphavantage
etc.);


In [57]:
yf.pdr_override()
x  = data.get_data_yahoo("ABEV3.SA") 
x['ticker'] = 'ABEV3.SA'

[*********************100%***********************]  1 of 1 completed


## Crie e armazene esses dados em um banco de dados SQL local ou em nuvem

In [56]:
engine = create_engine('sqlite:////content/teste.db', echo=False)
x.to_sql('data', con=engine)


5842